## Григорьев Дмитрий, вариант 3.
### Импортируем необходимые модули 
### + наш модуль.

In [1]:
from math import cos, sqrt
from mft2 import PolLagrange, bisection
from pandas import DataFrame

### Найти $\int_\limits{0}^1 \cos x \sqrt[4]{x}dx$

In [2]:
exact_value = 0.6539014401                 #"точное" значение
f = lambda x: cos(x)*x**(1/4)
a, b = 0.0, 1.0
numerical_values = [0]*4                   #Сюда будут записываться все результаты

### Вычисление методом средних прямоугольников по двум узлам

In [3]:
def middle_rectangles_integrate(f, a, b, n): #вычисляет приближенное значение методом средних прямоугольников по n узлам
    s = 0
    h = (b-a)/(n-1)
    for i in range(n-1):
        s+=f(a+(2*i+1)*h/2)
    return h*s

numerical_values[0] = (middle_rectangles_integrate(f,a,b,2)) # n=2 по условию
numerical_values[0]

0.7379560303827855

### Вычисление с помощью квадратурной формулы с весом $\rho=\sqrt[4]{x}$ по узлам $x_1 = \frac{1}{4},x_2 = \frac{3}{4}$.
### $\int_\limits{0}^1 \cos x \sqrt[4]{x}dx \approx A_1 \cos(\frac{1}{4}) + A_2 \cos(\frac{3}{4})$
### $A_1 =-2\int_\limits{0}^1 \sqrt[4]{x}(x-\frac{3}{4})dx = \frac{14}{45} $
### $A_2 =2\int_\limits{0}^1 \sqrt[4]{x}(x-\frac{1}{4})dx = \frac{22}{45} $

In [4]:
nodes = [1./4,3./4]
coeffs = [14./45, 22./45]
g = lambda x: cos(x)
def quadrature_with_weight(f,nodes,coeffs):
    s = 0
    for i in range(len(nodes)):
        s+=coeffs[i]*f(nodes[i])
    return s
numerical_values[1] = quadrature_with_weight(g, nodes, coeffs)
numerical_values[1]

0.6591539782038464

### Формула Гаусса: узлы $x_1 = -\frac{1}{2}\sqrt{\frac{3}{5}}+\frac{1}{2},x_2 = \frac{1}{2}, x_3 = \frac{1}{2}\sqrt{\frac{3}{5}}+\frac{1}{2}$

In [5]:
def Gauss_formula_3(f, a, b):           #функция для случая трёх узлов
    Legendre_nodes = [-sqrt(3./5), 0, sqrt(3./5)]
    coeffs = [5./9, 8./9, 5./9]
    s = 0
    k1, k2 = (b-a)/2, (b+a)/2
    for i in range(len(coeffs)):
        s+=coeffs[i]*f(k1*Legendre_nodes[i]+k2)
    return k1*s
numerical_values[2] = Gauss_formula_3(f, a, b)
numerical_values[2]

0.6581582678226776

### Формула типа Гаусса по трём узлам
### Узлы находим как корни ортогонального к $1,x,x^2$ относительно веса $\sqrt[4]{x}$ многочлена третьей степени $\omega(x)=x^3+a_1x^2+a_2x+a_3$
### Найдя коэффициенты системы,обеспечивающей ортогональность, из выражения $\mu_k = \int_\limits{0}^1 x^{1/4+k}dx = \frac{4}{5+4k}$, получаем эту самую систему:
### $\frac{4}{13}a_1+\frac{4}{9}a_2+\frac{4}{5}a_3 = -\frac{4}{17}$,
### $\frac{4}{17}a_1+\frac{4}{13}a_2+\frac{4}{9}a_3 = -\frac{4}{21}$,
### $\frac{4}{21}a_1+\frac{4}{17}a_2+\frac{4}{13}a_3 = -\frac{4}{25}$,
### откуда $a_1 = -\frac{39}{25}, a_2 = \frac{117}{175}, a_3 = -\frac{39}{595}$

### Найдём корни нашего многочлена через задачу обратного интерполирования -- интерполияционный многочлен по 4-ём узлам совпадёт с нашим.

In [6]:
P = lambda x: x**3 -39./25*x**2 + 117./175*x-39./595
nodes = [0, 1./3, 2.3 ,1]
values = [P(nodes[i]) for i in range(len(nodes))]
#вычислим его корни приближенно
polynom = PolLagrange(nodes, values)        #инициализуем интерполяционный многочлен Лагранжа
buf = bisection(polynom, 0, a, b, 10**(-7))

### Итак, узлы формулы:

In [7]:
roots = [buf[i][0] for i in range(len(buf))]
roots

[0.5262762431798255, 0.8944847431935001, 0.13923901362667382]

### Далее идёт функция, вычисляющая коэффициенты $A_k$. Например, $A_1 = \frac{\mu_2-(x_2+x_3)\mu_1+x_2x_3\mu_0}{(x_1-x_2)(x_1-x_3)}$

In [8]:
mu = [4./13, 4./9, 4./5]    #первые коэффициенты системы, которые используются в формуле
def get_coeffs(pivot, q, r, mu):
    return (mu[0]-(q+r)*mu[1]+mu[2]*q*r)/((pivot - q)*(pivot - r))

coeffs = [get_coeffs(roots[i%3],roots[(i+1)%3],roots[(i+2)%3],mu) for i in range(len(roots))] #собственно, сами коэффициенты
for i in range(len(roots)):
    numerical_values[3]+=coeffs[i]*g(roots[i])
numerical_values[3]

0.6539017460994131

### Итого:

In [9]:
DataFrame([[numerical_values[k] for k in range(4)],
           [exact_value - numerical_values[k] for k in range(4)]],
          index = ["Значение","Фактическая погрешность"], columns = ["Результат метода средних прямоугольников","Результат построенной интерполяционной формулы с заданным весом","Результат формулы Гаусса","Результат полученной формулы типа Гаусса"])

,Результат метода средних прямоугольников,Результат построенной интерполяционной формулы с заданным весом,Результат формулы Гаусса,Результат полученной формулы типа Гаусса
Значение,0.737956,0.659154,0.658158,6.539017e-01
Фактическая погрешность,-0.084055,-0.005253,-0.004257,-3.059994e-07
